### SQL

In [1]:
# Импортируем необходимые библиотеки
import pandas as pd
from sqlalchemy import create_engine
import psycopg2

In [2]:
# Подключаемся к базе
db_config = {'user': '****', # имя пользователя
             'pwd': '****', # пароль
             'host': '****',
             'port': ****, # порт подключения
             'db': '****'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                         db_config['pwd'],
                                                         db_config['host'],
                                                         db_config['port'],
                                                         db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'}, echo=False)

**Таблица `visits`**

In [3]:
query = '''
        select * from visits;

        '''

In [4]:
df_visits = pd.io.sql.read_sql(query, con=engine)

In [5]:
df_visits.head()

,id,uid,device,endts,sourceid,startts
0,0,16879256277535980062,touch,2017-12-20 17:38:00,4,2017-12-20 17:20:00
1,1,104060357244891740,desktop,2018-02-19 17:21:00,2,2018-02-19 16:53:00
2,2,7459035603376831527,touch,2017-07-01 01:54:00,5,2017-07-01 01:54:00
3,3,16174680259334210214,desktop,2018-05-20 11:23:00,9,2018-05-20 10:59:00
4,4,9969694820036681168,desktop,2017-12-27 14:06:00,3,2017-12-27 14:06:00


In [6]:
df_visits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 358532 entries, 0 to 358531
Data columns (total 6 columns):
id          358532 non-null int64
uid         358532 non-null object
device      358532 non-null object
endts       358532 non-null datetime64[ns]
sourceid    358532 non-null int64
startts     358532 non-null datetime64[ns]
dtypes: datetime64[ns](2), int64(2), object(2)
memory usage: 16.4+ MB


**Таблица `orders`**

In [7]:
query = '''
        select * from orders;

        '''
df_orders = pd.io.sql.read_sql(query, con=engine)
df_orders.head()

,id,buyts,revenue,uid
0,1,2017-06-01 00:10:00,$17.00,10329302124590727494
1,2,2017-06-01 00:25:00,$0.55,11627257723692907447
2,3,2017-06-01 00:27:00,$0.37,17903680561304213844
3,4,2017-06-01 00:29:00,$0.55,16109239769442553005
4,5,2017-06-01 07:58:00,$0.37,14200605875248379450


In [8]:
df_orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50415 entries, 0 to 50414
Data columns (total 4 columns):
id         50415 non-null int64
buyts      50415 non-null datetime64[ns]
revenue    50415 non-null object
uid        50415 non-null object
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 1.5+ MB


**Таблица `advertisment_costs`**

In [9]:
query = '''
        select * from advertisment_costs;

        '''
df_adv_cost = pd.io.sql.read_sql(query, con=engine)
df_adv_cost.head()

,id,sourceid,dt,costs
0,1,1,2017-06-01,$75.20
1,2,1,2017-06-02,$62.25
2,3,1,2017-06-03,$36.53
3,4,1,2017-06-04,$55.00
4,5,1,2017-06-05,$57.08


In [10]:
df_adv_cost.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2542 entries, 0 to 2541
Data columns (total 4 columns):
id          2542 non-null int64
sourceid    2542 non-null int64
dt          2542 non-null datetime64[ns]
costs       2542 non-null object
dtypes: datetime64[ns](1), int64(2), object(1)
memory usage: 79.5+ KB


**1. Посчитайте затраты на каждый рекламный источник.**

>Таблица **advertisment_costs** (информация о затратах на маркетинг):
* `sourceid` — идентификатор рекламного источника
* `dt` — дата
* `costs` — затраты на этот рекламный источник в этот день

In [11]:
query_1 = '''
        select
            sourceid, sum(costs) as costs_sum
        from 
            advertisment_costs as adv
        group by adv.sourceid;
        '''

In [12]:
costs_total = pd.io.sql.read_sql(query_1, con=engine)

In [13]:
costs_total

,sourceid,costs_sum
0,10,"$5,822.49"
1,9,"$5,517.49"
2,1,"$20,833.27"
3,5,"$51,757.10"
4,4,"$61,073.60"
5,2,"$42,806.04"
6,3,"$141,321.63"


Больше всего потратили на рекламный источник с номером `3`.

**2. Посчитайте суммарную выручку, максимальную и минимальную выручку от заказов, сделанных с настольных ПК (тип устройства desktop) пользователями, которые хотя бы раз перешли из рекламного источника, на который было потрачено больше всего денег. Сделайте выводы**

In [14]:
query_2 = '''
        SELECT
            MIN(revenue) AS revenue_min,
            MAX(revenue) AS revenue_max,
            SUM(revenue) AS total_revenue      
        FROM
            orders AS o
        WHERE o.uid IN (
                        SELECT DISTINCT uid 
                        FROM visits AS v
                        WHERE v.device='desktop' AND
                              v.sourceid=3
                        );
        '''
df_revenue = pd.io.sql.read_sql(query_2, con = engine)

In [15]:
df_revenue

,revenue_min,revenue_max,total_revenue
0,$0.00,"$2,633.28","$106,058.86"


>**Выводы**
* Источник рекламы `№ 3` на который потратили больше всего денег `141,321.63 $` принес `106,058.86 $` выручки
* самый "дорогой" заказ принес `2,633.28 $`
* самый "дешевый" не стоил ничего `0.00 $`

**3 Введём новую метрику эффективности рекламны - дневные затраты на рекламу на пользователя по источнику. Она рассчитывается для каждого отдельного дня и равна объёму денежных средств, которые были израсходованы на то, чтобы пользователь перешёл по рекламе на сайт из данного рекламного источника. Иными словами, метрика для каждого дня равна отношению рекламных затрат по данному источнику к количеству посетителей, перешедших на сайт с данного источника. Рассчитайте для каждого дня введённую метрику. Считайте, что целевое действие было произведено тогда, когда сесия началась. Например, если посетитель провёл на сайте несколько минут с 23:50 14 апреля по 0:20 15 апреля, то его нужно включать в рассчёт за 14 апреля**

In [16]:
query_3 = '''
        SELECT 
        adv_cost.dt AS date,
        adv_cost.sourceid,
        adv_cost.daily_cost,
        gr_visits.daily_users,
        adv_cost.daily_cost/gr_visits.daily_users AS ad_costs_per_user
        FROM 
        
        (SELECT
             dt,
             sourceid,
             SUM(costs) AS daily_cost
        FROM 
             advertisment_costs AS adc
        GROUP BY 
             adc.dt,
             adc.sourceid) AS adv_cost
        
        JOIN (SELECT
                    startts, sourceid, COUNT(uid) AS daily_users
              FROM (SELECT DATE_TRUNC('DAY', startts) AS startts, sourceid, uid 
                    FROM visits) AS subquery
              GROUP BY 
                  startts,
                  sourceid               
        ) AS gr_visits
        ON (adv_cost.sourceid = gr_visits.sourceid AND adv_cost.dt = gr_visits.startts)
        ORDER BY date, sourceid;
             
        '''
daily_costs = pd.io.sql.read_sql(query_3, con=engine)
daily_costs

,date,sourceid,daily_cost,daily_users,ad_costs_per_user
0,2017-06-01,1,$75.20,68,$1.10
1,2017-06-01,2,$132.56,85,$1.55
2,2017-06-01,3,$349.38,185,$1.88
3,2017-06-01,4,$76.74,141,$0.54
4,2017-06-01,5,$73.24,121,$0.60
5,2017-06-01,9,$13.06,52,$0.25
6,2017-06-01,10,$15.08,9,$1.67
7,2017-06-02,1,$62.25,60,$1.03
8,2017-06-02,2,$132.38,98,$1.35
9,2017-06-02,3,$314.22,180,$1.74


>**Вывод**
* Стоимость привлечения пользователя с каждого источника меняется каждый день  
* Источник `№ 3`стабильно дорогой
* Более выгодными представляются источники `№ 2` и `№ 4`
